## FINAL PROJECT DSCI 511 - Making an Impact in our Communities: PA

### TEAM MEMBERS

  - Weiying Wang (Rachel)
  - Man Tik Li (Justin)
  - Rodney McGhee (RJ)
  - Grace Sigalla


### PROJECT GOAL



*   For our project, we wanted to create a simplified database containing what we belive to be key information regarding PA counties. The information, such as poverty rate and employment rate, will be beneficial for the government and other non profit organizations that aim at creating developmental resources to help individuals in those communities. While the census website has all information needed, it could be very cumbersome to look for each information, especially for those without a tech background. Hence, our project aims at simplifiying that work, and pulling key information that will be beneficial for those looking to provide resources, or just trying to understand the economy of the communities.

*   We are collecting the data regarding PA counties from the census, specifically from their ACS 2019 survey (2020 data was not yet provided at the time we did our project) https://www.census.gov/programs-surveys/acs The data we chose to select are: county name, income rate, health coverage, employment and poverty percentage.

*   We also collected information from wikipedia, containing brief sumarry of PA counties that are represented in our project. 




## DISTRIBUTION


We have our project uploaded on Github from where it can be accessed at any point in time. 

Github link and distribution goes here: https://github.com/gn004422363/dsci511_final_project

## Libraries in Use

In [1]:
import requests
import mysql.connector 
from bs4 import BeautifulSoup
from tqdm import tqdm

## Data Acquisition

The Census Bureau conducts surveys on behalf of various federal government and local government agencies on topics such as employment, crime, health, consumer expenditures, and housing. 





*  First, we had to collect the data from the Census. 

*  Currently, the Census Bureau offers the public wider access to key U.S. statistics through the Census application programming interface (API) 

Here is an example of the request: 

```
requests.get("https://api.census.gov/data/2019/acs/acs1/profile?get=NAME," + Code_from_Census + "&for=county:*&in=state:42" + Key)
``

```
# "https://api.census.gov/data/2019/acs/acs1/profile?get=NAME"
```
The above URL represents the of survey we are pulling data from. ACS stands for the American Community Serivce Survey

Following the url, 
```
# "Code_from_Census"
```
This represents a question from the Census. In the code, we used a variable to store the question code so the code can easily switch between the different questions for our data points.

Immediatly after the code,
```
# "&for=county:*&in=state:42"
```
with this piece of code we can chose what area from the data comes from. In this code, we break the data down by county and state. Each county and state is represted by a number. We can grab every single county from a by state using the * character

Finally,

```
# "APIKEY"
```
*   We can access the API by signining up for a Key through their website. 

*    **Once we make the call, we can grab the data we need as a JSON Object**





In [2]:
def create_Request_Call():
    RJkey = "&key=0e7d419f16558d78e1ba8cd3a868e417d8f06239"
    incomebyfamily = "DP03_0086E"
    censusdata = requests.get("https://api.census.gov/data/2019/acs/acs1/profile?get=NAME," + incomebyfamily + "&for=county:*&in=state:42" + RJkey)
    
    return censusdata.json()

## Data Manipulation


### Web Scraping for the Wikipedia

We then grab data from the other survey by switching out the question code, and append it to the nested list.



```
#    censusIncomedata = requests.get("https://api.census.gov/data/2019/acs/acs1/profile?get=NAME," + incomebyfamily + "&for=county:*&in=state:42" + RJkey)
    censusIncomeData = censusIncomedata.json()
   
    for i, income in enumerate(CityNames):
        if counterForIncome > 40:
            break
        x = censusIncomeData[counterForIncome][1]
        CityNames[i].append(x)
```

We then have a nested list with the data we want to upload to the database!


Once we have our first pull, we parse through the document grabing the name of the county  

```
  for city in jsondocument:
        citynameaslist = []
        if cityNamecounter > 40:
            break
        cityNameString = jsondocument[cityNamecounter][0]
        citynameaslist.append(cityNameString)
        listofCities.append(citynameaslist)
        cityNamecounter = cityNamecounter + 1
```
Then storing into a nested list that looks like this
```
# [[County1],[County2],[County3], etc....]
```

In [3]:
def create_city_list(jsondocument):
    cityNameString = ''
    cityNamecounter = 1
    listofCities = []
    for city in jsondocument:
        citynameaslist = []
        if cityNamecounter > 40:
            break
        cityNameString = jsondocument[cityNamecounter][0]
        citynameaslist.append(cityNameString)
        listofCities.append(citynameaslist)
        cityNamecounter = cityNamecounter + 1
    
    return listofCities
        

We then use the parse list of county names and edit the names to fit the wikipedia articles url.

```
def getWikiData(CityNames):
    counterForWiki = 0
    pageURLpath = ""
    for i, cities in enumerate(CityNames):
        wikiData = ""
        if counterForWiki > 40:
            break
        cityName = CityNames[counterForWiki][0]
        urlReadyString = cityName.replace(" ", "_")
```

Then, we concatenate the wikipedia API call and the newly created county name url. 

```
API_path = "https://en.wikipedia.org/api/rest_v1/"
##Create URL
        pageURLpath = "page/summary/" + urlReadyString
        API_Path_With_Link = API_path + pageURLpath
    

```
Finally, we call the wikipedia API use the rest module to grab the "extract", append the data to each county list and return the data.



```
        r = requests.get(API_Path_With_Link)
        County = r.json()
        wikiData = County["extract"]
        CityNames[i].append(wikiData)
```

In [4]:
def getWikiData(CityNames):
    counterForWiki = 0
    pageURLpath = ""
    for i, cities in enumerate(CityNames):
        wikiData = ""
        if counterForWiki > 40:
            break
        cityName = CityNames[counterForWiki][0]
        urlReadyString = cityName.replace(" ", "_")
        API_path = "https://en.wikipedia.org/api/rest_v1/"
##Create URL
        pageURLpath = "page/summary/" + urlReadyString
        API_Path_With_Link = API_path + pageURLpath
        ##print(API_Path_With_Link)
    
##Call URL with request module
        r = requests.get(API_Path_With_Link)
        County = r.json()
        wikiData = County["extract"]
        CityNames[i].append(wikiData)
        counterForWiki = counterForWiki + 1
    return CityNames
    

We then grab data from the other survey by switching out the question code, and append it to the nested list.

```
#    censusIncomedata = requests.get("https://api.census.gov/data/2019/acs/acs1/profile?get=NAME," + incomebyfamily + "&for=county:*&in=state:42" + RJkey)
    censusIncomeData = censusIncomedata.json()
   
    for i, income in enumerate(CityNames):
        if counterForIncome > 40:
            break
        x = censusIncomeData[counterForIncome][1]
        CityNames[i].append(x)
```

Getting data for income:

In [5]:
def getDataforIncome(CityNames):
    counterForIncome = 1
    RJkey = "&key=0e7d419f16558d78e1ba8cd3a868e417d8f06239"
    incomebyfamily = "DP03_0086E"
    censusIncomedata = requests.get("https://api.census.gov/data/2019/acs/acs1/profile?get=NAME," + incomebyfamily + "&for=county:*&in=state:42" + RJkey)
    censusIncomeData = censusIncomedata.json()
   
    for i, income in enumerate(CityNames):
        if counterForIncome > 40:
            break
        x = censusIncomeData[counterForIncome][1]
        CityNames[i].append(x)
        counterForIncome = counterForIncome + 1
        
    return CityNames
        
      

Getting data for health coverage:

In [6]:
def getDataforHealthCareCoverage(CityNames):
    counterForHC = 1
    RJkey = "&key=0e7d419f16558d78e1ba8cd3a868e417d8f06239"
    healthcoverage = "DP03_0095PE"
    censusHealthCaredata = requests.get("https://api.census.gov/data/2019/acs/acs1/profile?get=NAME," + healthcoverage + "&for=county:*&in=state:42" + RJkey)
    censusHealthCareData = censusHealthCaredata.json()
   
    for i, healthcare in enumerate(CityNames):
        if counterForHC > 40:
            break
        x = censusHealthCareData[counterForHC][1]
        CityNames[i].append(x)
        counterForHC = counterForHC + 1
        
    return CityNames
        

Getting data for employment:

In [7]:
def getDataforEmployment(CityNames):
    counterForEmployment = 1
    RJkey = "&key=0e7d419f16558d78e1ba8cd3a868e417d8f06239"
    employmentOverSixteen = "DP03_0001E"
    censusEmploymentdata = requests.get("https://api.census.gov/data/2019/acs/acs1/profile?get=NAME," + employmentOverSixteen + "&for=county:*&in=state:42" + RJkey)
    censusEmploymentData = censusEmploymentdata.json()
   
    for i, employment in enumerate(CityNames):
        if counterForEmployment > 40:
            break
        x = censusEmploymentData[counterForEmployment][1]
        CityNames[i].append(x)
        counterForEmployment = counterForEmployment + 1
        
    return CityNames

Getting data for pverty:

In [8]:
def getDataforPoverty(CityNames):
    counterForPoverty = 1
    RJkey = "&key=0e7d419f16558d78e1ba8cd3a868e417d8f06239"
    povertyunder = "DP03_0134PE"
    censusPovertydata = requests.get("https://api.census.gov/data/2019/acs/acs1/profile?get=NAME," + povertyunder + "&for=county:*&in=state:42" + RJkey)
    censusPovertyData = censusPovertydata.json()
   
    for i, employment in enumerate(CityNames):
        if counterForPoverty > 40:
            break
        x = censusPovertyData[counterForPoverty][1]
        CityNames[i].append(x)
        counterForPoverty = counterForPoverty + 1

    return CityNames    

We then have a nested list with the data we want to upload to the database!

## Database

After we extracted and cleaned the data. We will store them into MySQL database. The SQL database was hosted from MYSQLWorkbrench.

To able to connect to the server. You will need the IP address (host) and port from MYSQLWorkbrench. Then, you will need to enter the username and the password to grant access to the SQL.

After that, we will need to create a database named "dsci511db" to store our table.

In [9]:
# This code create a database in MySQL
# Connect to server
cnx = mysql.connector.connect(
host="127.0.0.1",
port=3306,
user="root",
password=mypassword)

# Get a cursor
cur = cnx.cursor()

# Create database
cur.execute("DROP DATABASE IF EXISTS dsci511db")
cur.execute("CREATE DATABASE dsci511db")

# Check database
cur.execute("SHOW DATABASES")

# Print output
for i in cur:
    print(i)

# Close connection
cur.close()

('dsci511db',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


True

After we created the "dsci511db" database in MySQL. We will create a table named "census" to store the data we extracted from CityNames.

In the following code, we will need to add database="dsci511db" to tell the SQL we would like to select the dsci511db. Then, we will run the create table statement to specify the names of the columns of the table.

In [10]:
# Connect to server and select the database "dsci511db" we created from above
cnx = mysql.connector.connect(
host="127.0.0.1",
port=3306,
user="root",
password=mypassword,
database="dsci511db")

# Get a cursor
cur = cnx.cursor()

# Create table
cur.execute("DROP TABLE IF EXISTS census")
cur.execute("CREATE TABLE census (name VARCHAR(100), wiki VARCHAR(1000), income VARCHAR(10), healthCov VARCHAR(10), employment VARCHAR(10), poverty VARCHAR(10))")

# Check census table
cur.execute("DESCRIBE census")

# Print output
for i in cur:
    print(i)

# Execute a query

# Close connection
cur.close()

('name', b'varchar(100)', 'YES', '', None, '')
('wiki', b'varchar(1000)', 'YES', '', None, '')
('income', b'varchar(10)', 'YES', '', None, '')
('healthCov', b'varchar(10)', 'YES', '', None, '')
('employment', b'varchar(10)', 'YES', '', None, '')
('poverty', b'varchar(10)', 'YES', '', None, '')


True

The following code will inert the data into dsci511db by using insert statement:

In [11]:
def insert_to_db(data):
    # Connect to server
    db = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password=mypassword,
    database="dsci511db")

    # Get a cursor
    cur = cnx.cursor()
    
    # Insert data
    sql = "INSERT INTO census(name, wiki, income, healthCov, employment, poverty) VALUES (%s, %s, %s, %s, %s, %s)"
    
    for i in data:
        cur.execute(sql, i)
        
    db.commit()

This function will run the query and print out the result in format.

In [12]:
def get_queries():
    # Connect to server
    db = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password=mypassword,
    database="dsci511db")

    # Get a cursor
    cur = cnx.cursor()

    # Query
    query = "SELECT * FROM census"
    cur.execute(query)
    
    # Fetch the data and store in result
    result = cur.fetchall()
    
    # Bold
    start = "\033[1m"
    end = "\033[0;0m"
    
    # Output
    for (name, summary, income, health, employment, povert) in result:
            print(start + "Name =" + end, name)
            print(start + "Summary =" + end, summary)
            print(start + "Income in 2019 =" + end, income)
            print(start + "Health Coverage =" + end, health)
            print(start + "Employment =" + end, employment)
            print(start + "Percent in Povert =" + end, povert, "%\n")
            
    cur.close()

## Output

The output will print in following format:
1. Name
2. Wikidedia
3. Income in 2019
4. HealthCoverage
5. Employment
6. Percent in Povert

In [13]:
## MASTER FUNCTION

## create list to hold data
dictionaryofCities = []

CityNames = []

jsondocument = create_Request_Call()
CityNames = create_city_list(jsondocument)
CityNames = getWikiData(CityNames)
CityNames = getDataforIncome(CityNames)
CityNames = getDataforHealthCareCoverage(CityNames)
CityNames = getDataforEmployment(CityNames)
CityNames = getDataforPoverty(CityNames)
insert_to_db(CityNames)
get_queries()

Name = Cumberland County, Pennsylvania
Summary = Cumberland County is a county located in the U.S. state of Pennsylvania. As of the 2010 census, the population was 235,406. Its county seat is Carlisle.
Income in 2019 = 93456
Health Coverage = 245100
Employment = 207849
Percent in Povert = 6.4 %

Name = Allegheny County, Pennsylvania
Summary = Allegheny County is located in the southwest of the U.S. state of Pennsylvania. As of the 2020 census, the population was 1,250,578, making it the state's second-most populous county, following Philadelphia County. The county seat is Pittsburgh.
Allegheny County is included in the Pittsburgh, PA Metropolitan Statistical Area, and in the Pittsburgh Designated Market Area.
Income in 2019 = 90891
Health Coverage = 1203380
Employment = 1015496
Percent in Povert = 10.8 %

Name = Westmoreland County, Pennsylvania
Summary = Westmoreland County is a county located in the U.S. state of Pennsylvania. At the 2010 census, the population was 365,169. The count

## README

README
### Introduction

For our project, we wanted to create a simplified database containing what we believe to be key information regarding PA counties. The information, such as poverty rate and employment rate, will be beneficial for the government and other nonprofit organizations that aim at creating developmental resources to help individuals in those communities. While the census website has all information needed, it could be very cumbersome to look for each piece of information, especially for those without a tech background. Hence, our project aims at simplifying that work and pulling key information that will be beneficial for those looking to provide resources, or just trying to understand the economy of the communities.

In most cases, the best way to look up information is to google and use several sources to create a background of the community you work with. We decided to ask, "What if we create a tool that allows us to gather crucial information about a county in one accessible space?" The first Question we asked is "What information do we want to grab?The Census Bureau conducts surveys on behalf of various federal government and local government agencies on topics such as employment, crime, health, consumer expenditures, and housing.We want to use the data to create a tool for non-profit organizations, government agencies, communities to understand their communities better.


### Reach the goal

Using SQL, We created a database containing important census data and important Wikipedia.The database allows the interested parties to look up counties in Pennsylvania and find information about:A General description about the county from Wikipedia. Census Data including income, Healthcare Coverage, employment, and poverty level.

Our progress is: First, we had to collect the data from the Census.Currently, the Census Bureau offers the public wider access to key U.S. statistics through the Census application programming interface (API).We can access the API by signining up for a Key through their website.They offer several different surveys to access data.

Census application programming interface (API): We chose The American Community Survey (ACS) from 2019 to pull data.The American Community Survey (ACS) is an ongoing survey that provides data every year -- giving communities the current information they need to plan investments and services.2020’s data is unavailable, but will be released soon. 


### Team members

RJ and Grace worked on created the code that grabs the data from the Census and Wikipedia. 
Justin and Rachel created our interface for our database which stores our data.



### Data

First we get our key from the website:Each survey provides a code that represents a question from the Census.You then need to chose if you want to look at a state, county, or townships.We then use the get command to receive the data as a JSON.

We then Parsed through the JSON file and created lists to hold our data. 
Using tools like BeautifulSoup, we used the parsed data from the JSON file to web scrape Wikipedia. 
Following this, We then exchanged the question code to find the data we want and we parse and add the data to our list. 
The data looks something like this before we add it to our database:
[County, Wikipedia Summary, Income Data, Health Care, Employment in the county, and poverty level]


### Challenge that we faced

The Census data is released yearly, so we can only grab data from the surveys that the Census API allows us.
Not everyone fills out the Census, so These results are always estimates
Navigating through the Census API can be difficult since there is so many questions to go through.
After the first part of the coding finished, We formated the data we receive from the Census in a more user-friendly way. Furthermore, We tried to split the name into county and state.


### Instruction

You can simply run it in Jupyternotebook. Database required localhost, please follow the instruction below to complete the setup.

    For database:
        1. The hosting requred MySQL and MySQL Workbrench
        2. Create connection in MySQL Workbrench
        3. Gather the info (host ip, port, username, password)
        4. Paste it in coressponding row:
            db = mysql.connector.connect(
            host=host address,
            port=port number,
            user=myusername,
            password="mypassword")
        
### Output

    1.Name - str
    2.Wikipedia Summary - str
    3.Income Data -str
    4.How many people are currently have Healthcare coverage? -str
    5.How many people are employed? -str
    6.Percent of Individuals who are under the poverty line -str
    
Column information for dataset extracted from older Census data for further data analysis.
All we need is to change the state code to get data from other states and also add more questions by changing the Question code